In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import numpy as np
from haversine import haversine, Unit
from sklearn.neighbors import BallTree
import folium

# Berlin's polygon

In [ ]:
berlin_gdf = gpd.read_file("berlin_polygons/berlin_ortsteile.shp")

In [ ]:
whole_berlin = berlin_gdf.dissolve().explode()

In [ ]:
m = folium.Map(location=[52.502957, 13.398887], zoom_start=8, tiles='CartoDB positron')

folium.GeoJson(whole_berlin["geometry"]).add_to(m)

m#.save(outfile="berlin_points.html")

# Create linear spaced points

In [ ]:
# Use 751/911 for 50m
# Use 375/456 for 100m
latitude_linspace = np.linspace(whole_berlin.geometry.bounds.miny, whole_berlin.geometry.bounds.maxy, num=375,)
longitude_linspace = np.linspace(whole_berlin.geometry.bounds.minx, whole_berlin.geometry.bounds.maxx, num=456,)

point_0 = (latitude_linspace[0], longitude_linspace[0])
point_1 = (latitude_linspace[0], longitude_linspace[1])
point_2 = (latitude_linspace[1], longitude_linspace[0])

print('Space between points: {:.3f}m, {:.3f}m'.format(haversine(point_0, point_1), haversine(point_0, point_2)))

Generate points in a grid using previous linear spaces

In [ ]:
grid_points = []

for lat_i in latitude_linspace:
    for lon_i in longitude_linspace:
        grid_points.append( (lat_i[0], lon_i[0]) )

print('Total number of grid points:', len(grid_points))

Transform to geopandas object

In [ ]:
df = pd.DataFrame(grid_points, columns =['lat', 'lon'])
grid_points_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon,df.lat)).set_crs('epsg:4326')
grid_points_gdf.head()

# Intersect points and Berlin's geometry

In [ ]:
pip = grid_points_gdf.within(whole_berlin['geometry'].iloc[0])

In [ ]:
#creating a new gdf keep only the intersecting records
berlin_grid_points = grid_points_gdf.loc[pip].copy()
print('Total number of points:', berlin_grid_points.shape[0])

Plot results

In [ ]:
m = folium.Map(location=[52.502957, 13.398887], zoom_start=8, tiles='CartoDB positron')


berlin_neighborhood_grid_points.apply(lambda row: folium.CircleMarker(location=[row["lat"], row["lon"]],
                                                                      radius=1, fill_color='red', color='red').add_to(m), axis=1)
folium.GeoJson(whole_berlin["geometry"]).add_to(m)

<<
m#.save(outfile="berlin_points.html")

# Snap points to network

In [ ]:
import osmnx as ox
from IPython.display import IFrame

In [ ]:
G = ox.graph.graph_from_polygon(whole_berlin["geometry"].iloc[0], 
                                network_type='all_private', 
                                simplify=False, 
                                retain_all=True, 
                                truncate_by_edge=False, 
                                clean_periphery=True, 
                                custom_filter=None)

Undirected network graph

In [ ]:
uG = ox.utils_graph.get_undirected(G)

Projected network graph

In [ ]:
pG = ox.projection.project_graph(G, to_crs='epsg:4326')

Graph edges

In [ ]:
gdf_edges = ox.utils_graph.graph_to_gdfs(pG, nodes=False)[["geometry", "length"]]

In [ ]:
aa = berlin_grid_points
aa

In [ ]:
edges = ox.distance.nearest_edges(pG, aa['lon'], aa['lat'], interpolate=.001, return_dist=True)

In [ ]:
np.array(edges[1]).mean()

In [ ]:
lines = gdf_edges.loc[edges[0], "geometry"]

# Filter points that are too close together

In [ ]:
n_points = 5

points_on_graph = []
for line, (point_idx, point) in zip(lines.items(), aa.iterrows()):
    possible_points = line[1].interpolate(np.random.rand(n_points), normalized=True)
    
    distance_between_pts = point.geometry.distance(possible_points)
    point_on_graph = possible_points[distance_between_pts.argmin()]
 
    points_on_graph.append(point_on_graph)

Plot grid points and their projections on the network

In [ ]:
m = folium.Map(location=[52.502957, 13.398887], zoom_start=13, tiles='CartoDB positron')
m = folium.Map(location=[52.502957, 13.398887], zoom_start=13, tiles='openstreetmap')

#folium.GeoJson(lines).add_to(m)
aa.apply(lambda row: folium.CircleMarker(location=[row["lat"], row["lon"]],
                                         radius=1, fill_color='red', color='red').add_to(m), axis=1)
for i in range(len(points_on_graph)):
    folium.CircleMarker(location=[points_on_graph[i].y, points_on_graph[i].x],
                        radius=2, fill_color='green', color='green').add_to(m)

m#.save(outfile="berlin_points.html")

In [ ]:
x, y = [], []
for i in range(len(points_on_graph)):
    x.append(points_on_graph[i].x)
    y.append(points_on_graph[i].y)

df = pd.DataFrame({'lon': x, 'lat': y})

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lat'],df['lon']))

In [ ]:
gdf

In [ ]:
gdf = gdf.set_crs(4326)

In [ ]:
aa = gdf.geometry.apply(lambda x: gdf.distance(x).min())

In [ ]:
x = pd.DataFrame([df['lat'],df['lon']]).T
x = np.deg2rad(x)

ball_tree = BallTree(x, metric="haversine")

In [ ]:
x['index'] = x.index

Joing observations within a 20m radius of each other

In [ ]:
radius = 80
neighbors = ball_tree.query_radius(
    x[['lat', 'lon']],
    r=radius / 6371000, # meters over the earth radius in meters
    return_distance=False,  # choose whether you also want to return the distance
    sort_results=False,
)

In [ ]:
neighbors.shape

In [ ]:
codes, uniques = pd.factorize(pd.DataFrame(neighbors)[0].astype(str))

In [ ]:
df['panel'] = codes

In [ ]:
df2 = df.drop_duplicates(subset=['panel'], keep='first')

In [ ]:
df2

In [ ]:
m = folium.Map(location=[52.502957, 13.398887], zoom_start=13, tiles='CartoDB positron')

df2.apply(lambda row: folium.CircleMarker(location=[row["lat"], row["lon"]],
                                         radius=1, fill_color='red', color='red').add_to(m), axis=1)
folium.GeoJson(berlin_gdf["geometry"].iloc[0]).add_to(m)

m#.save(outfile="berlin_points.html")

Rename columns

In [ ]:
df2 = df2.rename(columns={"lon": "Longitude", "lat": "Latitude"})

In [ ]:
df2

Write points to file

In [ ]:
df2.to_csv('berlin_randompoints_onnetwork.csv', index=False)